# Sysbench - CPU Workloads: Firecracker microVM

In [1]:
import re
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## 1. Parse Sysbench Results

In [2]:
df = pd.DataFrame(columns=['Number of vCPUs',
                           'Number of Threads',
                           'Max Prime',
                           'Events per second',
                           'Total Time',
                           'Total Number of Events',
                           'Min Latency (ms)',
                           'Avg Latency (ms)',
                           'Max Latency (ms)',
                           '95th Percentile Latency (ms)',
                           'Total Execution Time across Threads (s)',
                           'Avg Number of Events per Thread',
                           'Avg Execution Time per Thread (s)'])

In [3]:
df

,Number of vCPUs,Number of Threads,Max Prime,Events per second,Total Time,Total Number of Events,Min Latency (ms),Avg Latency (ms),Max Latency (ms),95th Percentile Latency (ms),Total Execution Time across Threads (s),Avg Number of Events per Thread,Avg Execution Time per Thread (s)


In [4]:
for outputFile in os.listdir('./microVMOutputs'):
    linenum = 0
    filepath = os.path.join('./microVMOutputs', outputFile)
    if outputFile == '.gitkeep':
        continue
    with open(filepath, 'r') as f:
        testResult = [4]
        for line in f:
            if ':' in line:
                result = line.rsplit(':', 1)[1].strip()
                if 's' in result:
                    testResult.append(float(result[:-1]))
                elif '/' in result:
                    testResult.append(float(result.split('/')[0]))
                elif result != '':
                    testResult.append(float(result))
        df.loc[len(df)] = testResult


In [5]:
df = df.astype({'Number of vCPUs': int, 'Number of Threads': int, 'Max Prime': int})\
       .sort_values(by=['Number of Threads', 'Max Prime']).reset_index(drop=True)

df['Total Execution Time across Threads (s)'] = df['Total Execution Time across Threads (s)'] / 1000

In [6]:
df.head(20)

,Number of vCPUs,Number of Threads,Max Prime,Events per second,Total Time,Total Number of Events,Min Latency (ms),Avg Latency (ms),Max Latency (ms),95th Percentile Latency (ms),Total Execution Time across Threads (s),Avg Number of Events per Thread,Avg Execution Time per Thread (s)
0,4,1,10000,1313.00,30.0011,39398.0,0.73,0.76,6.93,0.78,29.96913,39398.0000,29.9691
1,4,1,50000,140.04,30.0008,4202.0,7.01,7.14,15.98,7.17,29.99060,4202.0000,29.9906
2,4,1,100000,53.02,30.0021,1591.0,18.62,18.85,26.85,18.95,29.99700,1591.0000,29.9970
3,4,1,500000,5.45,30.0770,164.0,181.70,183.38,201.22,189.93,30.07458,164.0000,30.0746
4,4,1,1000000,2.04,30.4351,62.0,486.69,490.84,504.31,502.20,30.43227,62.0000,30.4323
5,4,2,10000,2643.87,30.0007,79328.0,0.73,0.76,1.84,0.77,59.95418,39664.0000,29.9771
6,4,2,50000,280.56,30.0009,8418.0,7.02,7.13,13.87,7.17,59.98930,4209.0000,29.9946
7,4,2,100000,105.89,30.0076,3178.0,18.69,18.88,34.12,18.95,60.00146,1589.0000,30.0007
8,4,2,500000,10.91,30.1386,329.0,181.63,182.80,200.23,186.54,60.14204,164.5000,30.0710
9,4,2,1000000,4.09,30.3425,124.0,486.87,489.22,497.89,493.24,60.66367,62.0000,30.3318


In [7]:
df.to_csv('sysbench_microVM.csv')

## 2. Visualization

In [10]:
dff_100k = df.loc[df['Max Prime'] == 100000]
dff_1m = df.loc[df['Max Prime'] == 1000000]

plot1 = go.Scatter(x=dff_100k['Number of Threads'],
                   y=dff_100k['Events per second'],
                   mode='markers+lines',
                   name='Max Prime = 100,000')
 
plot2 = go.Scatter(x=dff_1m['Number of Threads'],
                   y=dff_1m['Events per second'],
                   mode='markers+lines',
                   name='Max Prime = 1,000,000')
 

xaxis_title="Number of Threads"
yaxis_title="Events per second"

fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
fig.append_trace(plot1, 1, 1)
fig.append_trace(plot2, 2, 1)
fig.update_yaxes(title_text=yaxis_title, row=1, col=1)
fig.update_yaxes(title_text=yaxis_title, row=2, col=1)
fig.update_xaxes(title_text=xaxis_title, row=1, col=1)
fig.update_xaxes(title_text=xaxis_title, row=2, col=1)
fig.add_vline(x=4, line_width=1, line_dash="dot", line_color="green")
fig.add_vline(x=2, line_width=1, line_dash="dot", line_color="green")
fig['layout'].update(height=600, width=1000,
                     title='CPU Speed (Eps) on a bare-metal server with 32 logical cores')
fig.show()

In [13]:
plot1 = go.Scatter(x=dff_100k['Number of Threads'],
                   y=dff_100k['Total Number of Events'],
                   mode='markers+lines',
                   name='Max Prime = 100,000')
 
plot2 = go.Scatter(x=dff_1m['Number of Threads'],
                   y=dff_1m['Total Number of Events'],
                   mode='markers+lines',
                   name='Max Prime = 1,000,000')
 

xaxis_title="Number of Threads"
yaxis_title="Total Number of Events"

fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
fig.append_trace(plot1, 1, 1)
fig.append_trace(plot2, 2, 1)
fig.update_yaxes(title_text=yaxis_title, row=1, col=1)
fig.update_yaxes(title_text=yaxis_title, row=2, col=1)
fig.update_xaxes(title_text=xaxis_title, row=1, col=1)
fig.update_xaxes(title_text=xaxis_title, row=2, col=1)
fig.add_vline(x=4, line_width=1, line_dash="dot", line_color="green")
fig.add_vline(x=2, line_width=1, line_dash="dot", line_color="green")
fig['layout'].update(height=600, width=1000,
                     title='Number of Events on a microVM with 4 logical cores')
fig.show()

In [12]:
plot1 = go.Scatter(x=dff_100k['Number of Threads'],
                   y=dff_100k['Avg Latency (ms)'],
                   mode='markers+lines',
                   name='Max Prime = 100,000')
 
plot2 = go.Scatter(x=dff_1m['Number of Threads'],
                   y=dff_1m['Avg Latency (ms)'],
                   mode='markers+lines',
                   name='Max Prime = 1,000,000')
 

xaxis_title="Number of Threads"
yaxis_title="Avg Latency (ms)"

fig = make_subplots(rows=2, cols=1, shared_xaxes=True)
fig.append_trace(plot1, 1, 1)
fig.append_trace(plot2, 2, 1)
fig.update_yaxes(title_text=yaxis_title, row=1, col=1)
fig.update_yaxes(title_text=yaxis_title, row=2, col=1)
fig.update_xaxes(title_text=xaxis_title, row=1, col=1)
fig.update_xaxes(title_text=xaxis_title, row=2, col=1)
fig.add_vline(x=4, line_width=1, line_dash="dot", line_color="green")
fig.add_vline(x=2, line_width=1, line_dash="dot", line_color="green")
fig['layout'].update(height=600, width=1000,
                     title='Average Latency for an Event by a Thread on a microVM with 4 logical cores')
fig.show()